<a href="https://colab.research.google.com/github/ypwang0408/AI_Final/blob/main/model2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = []
labels = []
classes = 43
cur_path = '/content/drive/MyDrive/AI_Final/archive'
print(os.listdir(cur_path))

In [ ]:
# Retrieving the images and their labels
for i in range(classes):
    print("load class", i, "....")
    path = os.path.join(cur_path, 'Train', str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '/' + a)
            image = image.resize((30, 30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [ ]:
# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)


# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42)

# Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
# Building the model

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 activation='relu', input_shape=(30, 30, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(43, activation='softmax'))

In [ ]:
# Compilation of the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=epochs, validation_data=(X_test, y_test))
model.save('/content/drive/MyDrive/AI_Final/model2.h5')

In [ ]:
# plotting graphs for accuracy
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# testing accuracy on test dataset

y_test = pd.read_csv('/content/drive/MyDrive/AI_Final/archive/Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data = []
cnt = 0
for img in imgs:
    if cnt % 1000 == 0:
      print(cnt)
    cnt += 1
    image = Image.open('/content/drive/MyDrive/AI_Final/archive/'+img)
    image = image.resize((30, 30))
    data.append(np.array(image))
    
X_test = np.array(data)

pred = np.argmax(model.predict(X_test),axis=1)

# Accuracy with the test data
print(accuracy_score(labels, pred))